In [60]:
import json
import os
import pandas as pd
import yaml
import re
PROJECT_FOLDER="src"

In [61]:
with open(f"../{PROJECT_FOLDER}/pbt_project.yml",'r') as yamlfile:
        cur_yaml = yaml.safe_load(yamlfile)

In [62]:
def extractJobDetails(pipeLineConfig):
    tmpList = [] 
    for key, entry in pipeLineConfig["processes"].items():
        configArray = {}
        configArray["Job Name"] = pipeLineConfig["metainfo"]["name"]
        configArray["Step Name"]  = entry["metadata"]["slug"]
        try: 
            configArray["Pipeline Object"] = entry["properties"]["pipelineId"]
            configArray["Pipeline Pretty"] = cur_yaml["pipelines"][entry["properties"]["pipelineId"]]["name"]
        except:
            configArray["Pipeline Object"] = ""
            configArray["Pipeline Pretty"] = ""
            print("ERROR "+entry["properties"]["pipelineId"])
        #job name
        #print(pipeLineConfig["metainfo"]["name"])
        #process step name
        #print(entry["metadata"]["slug"])
        #process step pipeline
        #print(entry["properties"]["pipelineId"])
        #config settings
        try:
            for key2, config in entry["properties"]["configurations"].items():
                if key2 == "schema":
                    for configValue in config:
                        configArray[configValue["name"]] = configValue["kind"]["value"]
                        #print(configValue["name"])
                        #print(configValue["kind"]["value"])
                        #print("\n")
                if key2 == "overrides":
                    for overKey, overEntry in config.items():
                        configArray[overKey] = overEntry
                        #print(overEntry)
                        #print("\n")
        except:
            pass
        tmpList.append(configArray)
        
    #print(tmpList)
    return tmpList

In [63]:
def extractPipelinePackageName():
    directory = f"../{PROJECT_FOLDER}/pipelines/"
    tmpList = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isdir(f):
            try:
                #print(f+"/code/setup.py")
                fo = open (f + "/code/setup.py", "r")
                setupFile = fo.read()
                pipelineName = re.findall("name = '(.*)',", setupFile)[0]
                packageName = re.findall("'main = (.*)\.",setupFile)[0]
                #print(re.findall("name = '(.*)',", setupFile))
                #print(re.findall("'main = (.*)\.",setupFile))
                tmpList.append({"packageWheelName": pipelineName ,"prettyName": cur_yaml["pipelines"]["pipelines/"+filename]["name"] , "pipeline":"pipelines/"+filename,"packageName":packageName})
                #pipeLineConfig = json.load(fo)
                #print(pipeLineConfig)
                #configList.extend(extractJobDetails(pipeLineConfig))
            except:
                print("ERROR:" + filename)
                pass
    return tmpList


def extractDatabricksDetails(pipeLineConfig):
    tmpList = [] 
    for item in databrickJobConfig["components"]:
         tmpList.append({"path":item["PipelineComponent"]["path"],"id":item["PipelineComponent"]["id"],"jobPretty": databrickJobConfig["request"]["name"]})
    #print(tmpList)
    return tmpList

def findInListByCriterion(test_list, search, criterion):
    res = None
    
    for sub in test_list:
        if sub[criterion] == search:
            res = sub
            break        
    return res

In [64]:
directory = f"../{PROJECT_FOLDER}/jobs/"
configList = []
databricksJobFiles = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isdir(f):
        try:
            fo = open (f + "/code/prophecy-job.json", "r")
            dbFo = open (f + "/code/databricks-job.json", "r")
            pipeLineConfig = json.load(fo)
            databrickJobConfig = json.load(dbFo)
             
            databricksJobFiles.extend(extractDatabricksDetails(databrickJobConfig))
            configList.extend(extractJobDetails(pipeLineConfig))
        except:
            pass
            print("ERROR: "+f)
        
 ## "-1.0-py3-none-any.whl"


#databricksJobFiles.extend(extractDatabricksDetails(databrickJobConfig))
df = pd.DataFrame(configList)

df3=pd.pivot_table(df.rename(columns={'Step Name': 'Entity'}).loc[df['Job Name'].str.contains('PIPELINE', case=False, regex=False, na=False)], columns=['sourceSystem'], index=['Entity'], aggfunc='size', fill_value=0).style 
df2 = pd.DataFrame(extractPipelinePackageName())
#df.to_excel("package_details.xlsx")
df4 =  df2[df2.duplicated(['packageName'])]



pipelinePackageName = extractPipelinePackageName()

#f = open("src/jobs/HM2_PIPELINE_JOB/code/prophecy-job.json", "r")
#pipeLineConfig = json.load(f)


ERROR: ../src/jobs/DEV_BBM_PIPELINE_JOB
ERROR: ../src/jobs/DEV_MTK_PIPELINE_JOB
ERROR: ../src/jobs/DEV_ANS_PIPELINE_JOB
ERROR: ../src/jobs/DEV_VST_PIPELINE_JOB
ERROR: ../src/jobs/DEV_LNX_PIPELINE_JOB
ERROR: ../src/jobs/DEV_JNS_PIPELINE_JOB


In [66]:
##Extract the generated databricks job filepaths
##Loop through each file and confirm it is going to exist in DBFS by looking at the current configuration for each pipeline and what it is going to build


##Inputs
## list of pipelines & their respective package details

compareJobWheelAndPipeWheel = []
for item in databricksJobFiles:
    packageDetails = findInListByCriterion(pipelinePackageName,item["id"],"pipeline")
    pipelineGeneratedWheelName = packageDetails["packageWheelName"]
    wheelNameInJob = item["path"].replace("dbfs:/FileStore/prophecy/artifacts/jnj/cp/__PROJECT_ID_PLACEHOLDER__/__PROJECT_RELEASE_VERSION_PLACEHOLDER__/pipeline/","").replace("-1.0-py3-none-any.whl","")
    if pipelineGeneratedWheelName!=wheelNameInJob :
        compareJobWheelAndPipeWheel.append({"Job Name": item["jobPretty"], "Pipeline Folder": item["id"], "Pipeline Wheel":packageDetails["packageWheelName"], "Job Wheel": wheelNameInJob, "Mismatched": True})
    else:
        compareJobWheelAndPipeWheel.append({"Job Name": item["jobPretty"], "Pipeline Folder": item["id"], "Pipeline Wheel":packageDetails["packageWheelName"], "Job Wheel": wheelNameInJob, "Mismatched": False})
        
    
#databricksJobFiles.extend(extractDatabricksDetails(databrickJobConfig))
df7 = pd.DataFrame(compareJobWheelAndPipeWheel)
df7[df7["Mismatched"]]

,Job Name,Pipeline Folder,Pipeline Wheel,Job Wheel,Mismatched
30,JOB_MD_SLS_DOC_PTNR_FUNC_HM2,pipelines/SAP_MD_SLS_DOC_PTNR_FUNC_HM2,SAP_MD_SLS_DOC_PTNR_FUNC_HMD,SAP_MD_SLS_DOC_PTNR_FUNC_HM2,True


In [67]:
df5 = pd.DataFrame(configList)


#df['C'] = df.apply(lambda x: x.source_system in x.Object, axis=1)
cleaned_column_names = (df5.columns
                        .str.strip()
                        .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
                        .str.lower()
                        .str.replace('[ _-]+', '_')
                        .str.replace('[}{)(><.!?\\\\:;,-]', ''))
df5.columns = cleaned_column_names
#df2.columns = ['job_name', 'object', 'pipeline_object', 'pipeline_pretty', 'source_system', 'target_schema', 'mandt', 'source_database', 'dai_etl_id', 'config_database', 'config_databse', 'config_database_2', 'dbtable1', 'dbtable2', 'dbtable3', 'dbtable4', 'dbtable5', 'dbtable6', 'source_table', 'source_table1', 'dbtable', 'divisor', 'stk_sgmnt', 'uom_from_srvc_ent_sht', 'logl_sys', 'qty_in_parel_unit_of_meas', 'goods_rcpt_blok_stk', 'type_of_parel_unit_of_meas', 'val_goods_rcpt_blok_stk', 'seasn_yr', 'seasn', 'fshn_clct', 'fshn_theme', 'qty3', 'char_val_1', 'char_val_2', 'char_val_3', 'tax_rptg_ctry_regn', 'orig_of_an_invc_itm', 'retn_amt_in_doc_crncy', 'retn_amt_in_co_cd_crncy', 'pstd_retn_amt_in_doc_crncy', 'pstd_scty_retn_amt', 'delv', 'delv_itm', 'exch_rt', 'mlt_acct_asgnmt', 'gr_ir_acct_clrng_val_lcl_crncy', 'qty_in_po_prc_unit', 'qty_in_valut_gr_blok_stk', 'amt_in_lcl_crncy', 'exch_rt_diff_amt', 'qty2', 'acc_at_orig', 'valut_goods_rcpt_blok_stk', 'amt_in_doc_crncy', 'pcdr_for_updt_sched_line_qty', 'dbtable7', 'dbname', 'tbname']
#df2['C'] = df.apply(lambda x: x.source_system in x.object, axis=1)
#df2["C"] = [x[0] in x[1] for x in zip(df2['object'], df2['source_system'])]
df5['SourceCheck'] = df5.apply(lambda x: str(x.source_system).upper() in str(x.job_name).upper(), axis=1)
df5['ObjectCheck'] = df5.apply(lambda x: str(x.step_name).upper() in str(x.pipeline_pretty).upper(), axis=1)
df5 = df5.loc[df5['job_name'].str.contains('PIPELINE', case=False, regex=False, na=False)]
df5 = df5[['SourceCheck', 'ObjectCheck'] + [x for x in df5.columns if x not in ['SourceCheck', 'ObjectCheck']]]
#df5.columns = ['SourceCheck', 'ObjectCheck', 'job_name', 'step_name', 'pipeline_object', 'pipeline_pretty', 'source_system', 'target_schema', 'mandt', 'source_database', 'dai_etl_id', 'config_database', 'config_database_2', 'config_databse', 'config_database_3', 'config_database_4', 'dbtable1', 'dbtable2', 'dbtable3', 'dbtable4', 'dbtable5', 'dbtable6', 'source_table', 'source_table1', 'dbtable', 'divisor', 'stk_sgmnt', 'uom_from_srvc_ent_sht', 'logl_sys', 'qty_in_parel_unit_of_meas', 'goods_rcpt_blok_stk', 'type_of_parel_unit_of_meas', 'val_goods_rcpt_blok_stk', 'seasn_yr', 'seasn', 'fshn_clct', 'fshn_theme', 'qty3', 'char_val_1', 'char_val_2', 'char_val_3', 'tax_rptg_ctry_regn', 'orig_of_an_invc_itm', 'retn_amt_in_doc_crncy', 'retn_amt_in_co_cd_crncy', 'pstd_retn_amt_in_doc_crncy', 'pstd_scty_retn_amt', 'delv', 'delv_itm', 'exch_rt', 'mlt_acct_asgnmt', 'gr_ir_acct_clrng_val_lcl_crncy', 'qty_in_po_prc_unit', 'qty_in_valut_gr_blok_stk', 'amt_in_lcl_crncy', 'exch_rt_diff_amt', 'qty2', 'acc_at_orig', 'valut_goods_rcpt_blok_stk', 'amt_in_doc_crncy', 'pcdr_for_updt_sched_line_qty', 'dbtable7', 'dbname', 'tbname']
df5 = df5.loc[~(df5['step_name'].isin(['DummyEntry']))]
df5

<ipython-input-67-ff846f6e8cd3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_column_names = (df5.columns
<ipython-input-67-ff846f6e8cd3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_column_names = (df5.columns
<ipython-input-67-ff846f6e8cd3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_column_names = (df5.columns


,SourceCheck,ObjectCheck,job_name,step_name,pipeline_object,pipeline_pretty,source_system,mandt,source_database,dai_etl_id,...,exch_rt_diff_amt,qty2,acc_at_orig,valut_goods_rcpt_blok_stk,amt_in_doc_crncy,pcdr_for_updt_sched_line_qty,dbtable7,bom_vld_to_dttm,dbname,tbname
0,True,True,ATL_PIPELINE_JOB,MD_SLS_ORDR_LINE,pipelines/SAP_01_MD_SLS_ORDR_LINE_ATL,SAP_01_MD_SLS_ORDR_LINE_ATL,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,ATL_PIPELINE_JOB,MD_SLS_DOC_PTNR_FUNC,pipelines/SAP_01_MD_SLS_DOC_PTNR_FUNC,SAP_01_MD_SLS_DOC_PTNR_FUNC,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,ATL_PIPELINE_JOB,MD_DELV_LINE,pipelines/SAP_MD_DELV_LINE_FSN_GEU_HCS_MRS_P01...,SAP_MD_DELV_LINE_FSN_GEU_HCS_MRS_P01_SVS_ATL,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,True,ATL_PIPELINE_JOB,MD_PO_SCHED_LINE_DELV,pipelines/SAP_01_MD_PO_SCHED_LINE_DELV,SAP_MD_PO_SCHED_LINE_DELV_ATL_BBA_BBL_BBN_BWI_...,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,True,ATL_PIPELINE_JOB,MD_SER_NUM_STOCK_SGMNT,pipelines/SAP_01_MD_SER_NUM_STOCK_SGMNT,SAP_MD_SER_NUM_STOCK_SGMNT_ATL_BBA_BBL_BBN_MRS,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,True,True,HCS_PIPELINE_JOB,MD_SLS_ORDR_LINE,pipelines/SAP_01_MD_SLS_ORDR_LINE_hcs,SAP_01_MD_SLS_ORDR_LINE_hcs,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,True,True,HCS_PIPELINE_JOB,MD_SER_NUM_STOCK_SGMNT,pipelines/SAP_02_MD_SER_NUM_STOCK_SGMNT,SAP_MD_SER_NUM_STOCK_SGMNT_HCS_MBP_P01_SVS,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,True,True,HCS_PIPELINE_JOB,MD_MATL_LOC,pipelines/SAP_MD_MATL_LOC_TAI_MRS_P01_HCS_BWI_...,SAP_MD_MATL_LOC_TAI_MRS_P01_HCS_BWI_GEU_SVS_MB...,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458,True,True,HCS_PIPELINE_JOB,MD_SLS_DOC_PTNR_FUNC,pipelines/SAP_01_MD_SLS_DOC_PTNR_FUNC,SAP_01_MD_SLS_DOC_PTNR_FUNC,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
directory = f"../{PROJECT_FOLDER}/pipelines/"
pipelinesWithMultiplePackages = []
for pipelineFolder in os.listdir(directory):
    f = os.path.join(directory, pipelineFolder)
    # checking if it is a file
    if os.path.isdir(f):
        #try:
            i=0
            for pipeLineFile in os.listdir(f+"/code"):
#                 if pipelineFolder == "SAP_01_MD_SLS_ORDR_LINE_bwi":
#                    print(pipeLineFile)
                if not pipeLineFile == ".DS_Store" and not pipeLineFile == "prophecy" :
                    i+=1 
                
            if i != 7:
                pipelinesWithMultiplePackages.append({"pipelineFolder":pipelineFolder, "extraPackages":i-7})
                #print(pipelineFolder + ":" + str(i))
            else:
                pass
                #print("OK------" + pipelineFolder + ":" + str(i))
#         except:
#             print(f)
#             pass
df8 = pd.DataFrame(pipelinesWithMultiplePackages)
df8

,pipelineFolder,extraPackages
0,SAP_MD_MATL_LOC_BBA_BBL_BBN_02,1
1,MD_PRCH_DELV_CNFRMS_1,1
2,MD_PRCH_DELV_CNFRMS_6,1
3,MD_PRCH_DELV_CNFRMS_8,1
4,JDE_MD_MATL_LOC_JET,1
5,JDE_MD_MATL_LOC_DJD,1
6,JDE_MD_MATL_LOC_JES,1
7,SAP_MD_MATL_LOC_TAI_MRS_P01_HCS_BWI_GEU_SVS_MB...,1
8,SAP_01_MD_PO_SCHED_LINE_DELV,1
9,JDE_MD_MATL_LOC_DEU,1


In [70]:
with pd.ExcelWriter("release_summary.xlsx") as writer:
    df3.to_excel(writer, sheet_name="Release Summary", index=True)
    df.to_excel(writer, sheet_name="Job Details", index=False)
    
    df2.to_excel(writer, sheet_name="Pipeline Details", index=False)
    df4.to_excel(writer, sheet_name="Dupe Packages", index=False)
    df5.to_excel(writer, sheet_name="Job Check", index=False)
    df7.to_excel(writer, sheet_name="WheelCheck", index=False)
    df8.to_excel(writer, sheet_name="ExtraPackage", index=False)